In [13]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../utils/")

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, MaxAbsScaler, RobustScaler

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [14]:
# include_see_also=False used for validation below.
# in deployment, include_see_also should be set to True
gc = GraphCreator("Sonata form", include_see_also=False) 
print("Number of Links to Search:", len(gc.next_links), "\n\n")
print(list(gc.primary_nodes.keys()), "\n\n")
print(gc.see_also_articles)

Number of Links to Search: 1023 


['Movement (music)', 'Exposition (music)', 'Musical development', 'Musical form', 'String quartet', 'Sonata', 'Coda (music)', 'Concerto', 'Symphony', 'Tonality', 'Recapitulation (music)', 'Introduction (music)', 'Classical music era', 'Musical analysis'] 


['Closely related key', 'Slow movement (music)', 'Sonata rondo form']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [15]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [16]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [17]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Wolfgang Amadeus Mozart,Ludwig van Beethoven,23
1,Ludwig van Beethoven,Wolfgang Amadeus Mozart,23
2,Charles-Valentin Alkan,Camille Saint-Saëns,22
3,Camille Saint-Saëns,Charles-Valentin Alkan,22
4,Camille Saint-Saëns,Claude Debussy,21


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [18]:
features_df = gc.get_features_df(rank=False)

# Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [19]:
gc.rank_similarity()
gc.features_df.sort_values("similarity_rank", ascending=False).head(10)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
509,Sonata cycle,147,2,69,78,0.097826,0.001740,0.000070,0.022731,1.0,1.0,0.096639,0,1.715754
247,Musical development,292,1,222,70,0.196210,0.007039,0.000597,0.016087,1.0,1.0,0.209302,1,1.509386
66,Sonata form,1013,1,714,299,1.000000,0.030556,0.001272,0.065046,0.0,0.0,1.000000,0,1.283967
101,Tonality,646,1,492,154,0.063158,0.008942,0.001285,0.020633,1.0,1.0,0.053275,1,1.025746
2388,Slow movement (music),25,1,11,14,0.021635,0.000561,0.000026,0.000699,1.0,1.0,0.008345,0,0.983291
607,Introduction (music),109,0,58,51,0.078597,0.001829,0.000286,0.015387,1.0,1.0,0.079720,1,0.932275
559,Arch form,127,1,70,57,0.059579,0.001584,0.000355,0.016436,1.0,1.0,0.066667,0,0.894758
522,Period (music),141,1,75,66,0.065896,0.001840,0.000474,0.016087,1.0,1.0,0.069106,0,0.883470
530,Progressive tonality,137,1,39,98,0.063927,0.002240,0.000147,0.011191,1.0,1.0,0.040055,0,0.882574
80,Musical form,875,1,459,416,0.069314,0.011471,0.001703,0.039517,1.0,1.0,0.075160,1,0.880265


# Scaling Features

We can easily scale our each of our features through the `scale_features_df` method. It will default to `Standard Scaler`, but we can specify alternate scalers in the `scaler` parameter.  

In [20]:
scaled_feature_df = gc.scale_features_df(scaler=MinMaxScaler, copy=True) # Makes a copy of the df
scaled_feature_df.sort_values("similarity_rank", ascending=False).reset_index().drop("index", axis=1)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
0,Sonata cycle,0.009240,1.0,0.004511,0.040562,0.096802,5.275133e-03,0.002396,0.118613,0.166667,0.25,0.096639,0.0,1.000000e+00
1,Musical development,0.018417,0.5,0.014514,0.036401,0.195297,2.134404e-02,0.021148,0.083942,0.166667,0.25,0.209302,1.0,8.797049e-01
2,Sonata form,0.064047,0.5,0.046679,0.155486,1.000000,9.265721e-02,0.045156,0.339416,0.000000,0.00,1.000000,0.0,7.483047e-01
3,Tonality,0.040820,0.5,0.032165,0.080083,0.062095,2.711487e-02,0.045618,0.107664,0.166667,0.25,0.053275,1.0,5.977834e-01
4,Slow movement (music),0.001519,0.5,0.000719,0.007280,0.020524,1.701423e-03,0.000855,0.003650,0.166667,0.25,0.008345,0.0,5.730353e-01
5,Introduction (music),0.006835,0.0,0.003792,0.026521,0.077551,5.546259e-03,0.010103,0.080292,0.166667,0.25,0.079720,1.0,5.432974e-01
6,Arch form,0.007974,0.5,0.004576,0.029641,0.058512,4.804257e-03,0.012562,0.085766,0.166667,0.25,0.066667,0.0,5.214284e-01
7,Period (music),0.008860,0.5,0.004903,0.034321,0.064836,5.578870e-03,0.016785,0.083942,0.166667,0.25,0.069106,0.0,5.148484e-01
8,Progressive tonality,0.008607,0.5,0.002550,0.050962,0.062864,6.793252e-03,0.005132,0.058394,0.166667,0.25,0.040055,0.0,5.143256e-01
9,Musical form,0.055313,0.5,0.030008,0.216329,0.068257,3.478286e-02,0.060496,0.206204,0.166667,0.25,0.075160,1.0,5.129799e-01


___
# Validation

Here, we _validate_ our results. For many articles, we already have some user defined links that are highly related to the present article. These are found in the **See Also** section of several Wikipedia articles (some pages do not have them). These are not ordered in importance in any meaningful way, and there are no rating scores.

The intuition in this validation is as follows: 

> _Given that we know some articles are highly related from user input, if the recommendations provided by this system are valid, we would expect to see those **See Also** links ranked relatively high on our list._ 

_Note: This validation is not meant as **confirmation** or **evaluation** of the results. It only provides us one way of telling if the results we are seeing are reasonably valid. It is important to note that we cannot compare these results across two different articles, as those would be two entirely different network structures, likely with different human labeled links._  


In [21]:
evaluate_metrics(scaled_feature_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry", "jaccard_similarity"], 
                 targets=gc.see_also_articles)

Metric Score,score,max score possible,difference,total targets,% targets in top 1%,% targets in top 5%,% targets in top 10%,% targets in top 20%
similarity_rank,0.994246,0.999973,0.005726,3.0,1.000000,1.000000,1.0,1.0
centrality,0.939746,0.999973,0.060227,3.0,0.333333,0.666667,1.0,1.0
adjusted_reciprocity,0.945824,0.999973,0.054149,3.0,0.666667,0.666667,1.0,1.0
page_rank,0.968694,0.999973,0.031279,3.0,0.666667,1.000000,1.0,1.0
shortest_path_length_from_entry,0.000632,0.999973,0.999341,3.0,0.000000,0.000000,0.0,0.0
jaccard_similarity,0.966653,0.999973,0.033320,3.0,0.333333,1.000000,1.0,1.0


The chart generated above compares different ranking metrics (left index) for a given article. The most important column, `score`, provides a fast way for us to compare these different metrics. 

For example, if we see a _score_ of 0.98 for a given ranking metric, The following statement would be true:

> All of the human labeled **See Also** links are present within the top 98% of our recommendations. 

Since the human labeled links comprise a range, it is not possible to get a score of 100%. The `max score possible` column indicates the score that would be achieved if all the human labeled _See Also_ links appeared at the top of our recommendations without any other links intervening.   

The `difference` column is an alternative way of looking at the score. If we had a 0.02 in this column, we could say:

> All the human labeled **See Also** links are contained within the top 2% of our recommendations. 

`Total targets` is the number of human labeled _See Also_ links. 

Because it is possible that different metrics could have similar scores, we want a way to break down the dispersion of the known related links to see if one metric does perform better than another. The trailing four columns provide us with a course way of measuring this dispersion. 

Each of these columns indicates the percentage of human labeled _See Also_ links captured within a given percentage of the top of our recommendations. For example, if we see a 0.92 in the `% targets in the top 1%` columns, we could say:

> 92% of the human labeled **See Also** links appear in the top 1% of our recommendations. 

The value of these columns is a follows - If two ranking metrics have similar scores, we _might_ consider the better performing one to be the one in which the majority of the human labeled links are higher in our recommendation list. 

___

In [22]:
gc.features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110717 entries, 0 to 110716
Data columns (total 14 columns):
node                                 110717 non-null object
degree                               110717 non-null int64
category_matches_with_source         110717 non-null int64
in_edges                             110717 non-null int64
out_edges                            110717 non-null int64
shared_neighbors_with_entry_score    110717 non-null float64
centrality                           110717 non-null float64
page_rank                            110717 non-null float64
adjusted_reciprocity                 110717 non-null float64
shortest_path_length_from_entry      110717 non-null float64
shortest_path_length_to_entry        110717 non-null float64
jaccard_similarity                   110717 non-null float64
primary_link                         110717 non-null int64
similarity_rank                      110717 non-null float64
dtypes: float64(8), int64(5), object(1)
memory

In [26]:
scaled_feature_df.sort_values("similarity_rank", ascending=False).reset_index()[['node']]

,node
0,Sonata cycle
1,Musical development
2,Sonata form
3,Tonality
4,Slow movement (music)
5,Introduction (music)
6,Arch form
7,Period (music)
8,Progressive tonality
9,Musical form
